# Data gathering and cleaning

## Before running the LDA model, we'll need to get the data into a clean, useable format:

- Get dataframe of texts from Academy urls 
- Create corpus(text must be combined per url, for now). df(url:combined_content)
- Perform first round of data cleaning
    - unwanted symbols
    - make lowercase
    - remove numbers
- Second round of cleaning
    - lemmatisation
    - stemming
    - remove stop words

## 1. Import Academy texts dataframe 

In [ ]:
import pandas as pd
import pickle

In [ ]:
df = pd.read_csv('../04_Data/processed_posts.csv', index_col=0)

In [ ]:
df.head(30)

In [ ]:
df.info()

## 2. Create corpus

We will use the initial setup (looking at specific html tags) as well as the per article analysis (which is more useful for EDA). For that we need to drop the 'tag', 'title' and 'published' column and combine rows with the same url.

### 2.1 Corpus with articles separated by tags

In [ ]:
# Keeping important columns

df_separated = df[['url','content']]

In [ ]:
df_separated

In [ ]:
# Everything looks good so far, let's just pick a random article
# and check that everything is okay

df_separated.loc[45].content

In [ ]:
with open('pkl_cellar/corpus_separated.pkl', 'wb') as ds:
    pickle.dump(df_separated, ds, protocol=pickle.HIGHEST_PROTOCOL)

### 2.2 Corpus with articles as a whole
This is mostly for EDA, to make sure the cleaned data offers valid results

In [ ]:
# Combining rows with the same url
# Should only have 159 rows

df_combined = df_separated.groupby(['url'])['content'].apply(' '.join).reset_index()

In [ ]:
df_combined

In [ ]:
df_combined.loc[45].content

In [ ]:
# Let's pickle and move on

with open('pkl_cellar/corpus_combined.pkl', 'wb') as dc:
    pickle.dump(df_combined, dc, protocol=pickle.HIGHEST_PROTOCOL)

## 3. Data cleaning round 1!

Converting to lower case, get rid of punctuation and numbers

In [ ]:
test = df_separated.loc[45].content

In [ ]:
import re
import string

In [ ]:
def cleaning_round1(text):
    '''lowercase, remove punctuation, remove \xa0, remove numbers + words with numbers'''
    
    text = text.lower()
    text = re.sub('-', ' ', text)
    
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\xa0', ' ', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[”“–‘’]', '', text )
    
    return text

In [ ]:
test_round1 = cleaning_round1(test)
test_round1

## 4. Data cleaning round 2!

The big guns are coming out: lemmatisation, tokenization, stopword removal. (perhaps bigrams and trigrams at later stage)

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import spacy

### 4.1 Using lemmatization to reduce words to their root words

In [ ]:
# lemmatisation

def lemmatizer(text):
    nlp = spacy.load('en_core_web_sm')

    text_out = []
    tokens = nlp(text)
    text_out = [" ".join(token.lemma_ for token in tokens)]
    
    text_out = [re.sub('-PRON-', 'i', str(text)) for text in text_out]
    
    return "".join(text_out)

In [ ]:
text_lem = lemmatizer(test_round1)
text_lem

### 4.2 Tokenizing (separating words)

In [ ]:
# tokenize texts

def tokenize(text):
    
    words = word_tokenize(text)
    
    return words

In [ ]:
text_tkn = tokenize(text_lem)

In [ ]:
text_tkn

### 4.3 Removing stop words from tokenized list

In [ ]:
# remove stopwords

def remove_stopwords(text):
    '''Takes tokenized (list) of words and removes stopwords'''
    
    stp_words = set(stopwords.words('english'))
    
    no_stp_words = [word for word in text if word not in stp_words]

    return ' '.join(no_stp_words)

In [ ]:
text_no_sw = remove_stopwords(text_tkn)

In [ ]:
text_no_sw

## 5. Running all texts through cleaner

- Step 1: Create one function to clean a test list
- Step 2: Test it!
- Step 3: Run entire set through cleaner
- Step 4: Pickle it!

In [ ]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [ ]:
test_df = df_separated.loc[[6, 37, 28]]

In [ ]:
test_df

In [ ]:
def nlp_cleaner(df):
    
    df['content'] = df['content'].apply(cleaning_round1)
    
    # cleaning round 2
    df['content'] = df['content'].apply(lemmatizer)
    df['content'] = df['content'].apply(tokenize)
    df['content'] = df['content'].apply(remove_stopwords)
    
    return df

In [ ]:
nlp_cleaner(test_df)

### 5.1 Everything works, time to take the entire df_separated to the cleaners
... then it's pickle time

In [ ]:
# needs about an hour to run

#lda_df = nlp_cleaner(df_separated)

In [ ]:
with open('pkl_cellar/corpus_processed.pkl', 'rb') as file:
    lda_df = pickle.load(file)

lda_df

### 5.2 Now using nlp_cleaner( ) on df_combined

In [ ]:
# takes about 5 minutes to run

lda_combined_articles = nlp_cleaner(df_combined)

In [ ]:
lda_combined_articles

In [ ]:
# with open('pkl_cellar/corpus_processed_full_articles.pkl', 'wb') as fa:
#    pickle.dump(lda_combined_articles, fa, protocol=pickle.HIGHEST_PROTOCOL)